# general

In [ ]:
# imports
import os
import copy
from datetime import date
from bidi.algorithm import get_display  # flips letters order - a problem that comes up in Hebrew

import numpy as np
import pandas as pd

from scipy.stats import pearsonr, spearmanr

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib import rcParams

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"  # allows to see all outputs of a cell rather then just from the last line
from IPython.display import Image   # allows to display existing images in the notebook

In [ ]:
rcParams['savefig.format'] = 'png'#'svg'#
sns.set(font_scale=1.3, style='white')

In [ ]:
# paths
cities_file = '/net/mraid08/export/jafar/Microbiome/Analyses/saar/corona/yeshuvim_20200301.csv'
questions_file = '/home/saarsh/Develop/Git/corona/data/Processed/forms/all_forms.csv'
patients_MOH_file= '/net/mraid08/export/jafar/Microbiome/Analyses/saar/corona/MOH/MOH_combined.csv'

In [ ]:
# create a hebrew-english cities dictionary
cities_df = pd.read_csv(cities_file, skiprows=1, encoding='hebrew').rename(columns={'שם_ישוב': 'city_he', 'שם_ישוב_לועזי': 'city_en'})
cities_df.columns = map(str.lower, cities_df.columns)
cities_df['city_he'] = cities_df['city_he'].str.strip()
cities_df['city_en'] = cities_df['city_en'].str.strip()

city_he2en_dict = cities_df.set_index('city_he')['city_en'].to_dict()
city_en2he_dict = cities_df.set_index('city_en')['city_he'].to_dict()

In [ ]:
# reading and cleaning files
questions_df = pd.read_csv(questions_file)
questions_df.columns = map(str.lower, questions_df.columns)
questions_df['date_only'] = pd.to_datetime(questions_df['timestamp']).dt.date
questions_df['city_he'] = questions_df['city_en'].map(city_en2he_dict)

patients_MOH_df = pd.read_csv(patients_MOH_file)

In [ ]:
# sanity check
questions_df.shape
questions_df['date_only'].min()
questions_df['date_only'].max()

In [ ]:
# parameters
minimal_population = None
minimal_answers = 20
minimal_days = 3
days_back = None
alpha = 0.05

hebrew = False
time = 'date_only'
location = 'city_en' if not hebrew else 'city_he'
area = 'district_en'

In [ ]:
# filtering - questions
questions_filtered_df = questions_df
if days_back:
    questions_filtered_df = questions_filtered_df[questions_filtered_df[time] > questions_filtered_df[time].max() - pd.Timedelta(days=days_back)]
if minimal_answers:
    questions_filtered_df = questions_filtered_df.groupby([location, time]).filter(lambda x: x.shape[0] > minimal_answers)
if minimal_days:
    questions_filtered_df = questions_filtered_df.groupby([location]).filter(lambda x: len(np.unique(x[time])) > minimal_days)
    
# filtering - patients
patients_MOH_filtered_df = patients_MOH_df.drop_duplicates(location)
if minimal_population:
    patients_MOH_filtered_df = patients_MOH_filtered_df[patients_MOH_filtered_df['population'] > minimal_population]

In [ ]:
# parameters lists
# srt - from confirmed patients
# srs - from entire population (iclandic paper)
symptoms_to_ignore = ['symptom_well', 'symptom_ratio',  'symptom_ratio_weighted']
all_symptoms = questions_filtered_df.columns[['symptom_' in col and col not in symptoms_to_ignore for col in questions_filtered_df.columns]].values.tolist()
all_conditions = questions_filtered_df.columns[['condition_' in col for col in questions_filtered_df.columns]].values.tolist()

In [ ]:
symptoms = ['symptom_ratio_weighted']

# patients symptoms correlation

In [ ]:
patients_col = [date(2020, 4, 1), date(2020, 4, 3)]

In [ ]:
def patients_symptoms_correlation_plot(symptoms, patients_col, start_date=None, end_date=None, corr_only=False, df_only=False):
    
    # data - patients
    p_df = patients_MOH_filtered_df.set_index([location])
    if type(patients_col) is list:
        p_df = (p_df[str(patients_col[1])].subtract(p_df[str(patients_col[0])]).divide(p_df['population']).dropna())*1000
        xlabel = 'new patients per 1000 residences' if not hebrew else get_display('חולים חדשים ל-1000 תושבים')
    else:
        p_df = (p_df[str(patients_col)].divide(p_df['population']).dropna())*1000
        xlabel = 'patients per 1000 residences' if not hebrew else get_display('חולים ל-1000 תושבים')

    # data - questions
    q_df = questions_filtered_df[[location, time] + symptoms]
    if start_date is not None and end_date is not None:
        q_df = q_df[q_df[time] >= start_date]
        q_df = q_df[q_df[time] <= end_date]
        q_df = q_df.groupby([location]).filter(lambda x: len(np.unique(x[time])) > (end_date-start_date).days)
    q_df = q_df.drop(time, axis=1)
    if len(symptoms) == 1:
        q_df = q_df.groupby([location])[symptoms[0]].mean()*100
#         q_df = q_df.groupby([location])[symptoms[0]].mean()
    else:
        q_df = (q_df.groupby([location])[symptoms].mean()*100).mean(axis=1)
#         q_df = (q_df.groupby([location])[[symptoms].mean()).mean(axis=1)

    # data - sync
    overlpped_cities = set(p_df.index) & set(q_df.index)
    p_df = p_df.loc[overlpped_cities]
    q_df = q_df.loc[overlpped_cities]
    
    if df_only:
        return q_df, p_df

    r_p, p_p = pearsonr(p_df, q_df)
    r_s, p_s = spearmanr(p_df, q_df)

    if corr_only:
        return r_p, p_p, r_s, p_s
    
    # plot
    fig = plt.figure(figsize=(14, 12))
    gs = fig.add_gridspec(1, 8)
    axes = [np.nan, np.nan]
    axes[0] = fig.add_subplot(gs[0, :-1])
    axes[1] = fig.add_subplot(gs[0, -1:])
    
    # plot the same on both axes
    for i in np.arange(len(axes)):
        axes[i] = sns.scatterplot(p_df, q_df, 
                                  hue=q_df, palette='Reds', edgecolor='gray', linewidth=1,
                                  size=patients_MOH_filtered_df.set_index(location).loc[overlpped_cities, 'population'], sizes=(500, 2000), ax=axes[i])
    
        for city in overlpped_cities:
            if hebrew:
                city = get_display(city)
            axes[i].annotate(city, (p_df.loc[city], q_df.loc[city]))
    
    # hide the spines between the axes
    axes[0].spines['right'].set_visible(False)
    axes[1].spines['left'].set_visible(False)
    axes[1].tick_params(labelleft=False)
    axes[1].set_ylabel('')
    
    # zoom-in / limit the view to different portions of the data
    axes[0].set_xlim(0, 0.45)  # outliers only
    axes[1].set_xlim(1.7, 1.75)  # most of the data
    axes[0].set_xticks(np.arange(axes[0].get_xlim()[0], axes[0].get_xlim()[1], 0.05))
    axes[1].set_xticks(np.arange(axes[1].get_xlim()[0], axes[1].get_xlim()[1], 0.05))

    # diagnoal lines between axes
    d = .015  # size of diagonal lines in axes coordinates
    kwargs = dict(transform=axes[0].transAxes, color='k', clip_on=False)
    axes[0].plot((1, 1), (-d, +d), **kwargs)
    axes[0].plot((1, 1), (1 - d, 1 + d), **kwargs)

    kwargs.update(transform=axes[1].transAxes, color='k', clip_on=False)
    axes[1].plot((0, 0), (-d, +d), **kwargs)
    axes[1].plot((0, 0), (1 - d, 1 + d), **kwargs)
                
    # only for the main axes
    axes[0].set_xlabel(xlabel)
    axes[0].set_ylabel('% symptoms in the city') if not hebrew else ax1.set_ylabel(get_display('אחוז הסימפטומים בעיר'))
#     axes[0].set_ylabel('mean symptoms ratio in the city') if not hebrew else ax1.set_ylabel(get_display('יחס סימפטומים ממוצע בעיר'))

    if len(symptoms) == 1:
        title = 'correlation between patients and {}'.format(symptoms[0]) if not hebrew else get_display('קוראלציה בין חולים ו- ') + symptoms[0]
    else:
        title = 'correlation between patients and symptoms'  if not hebrew else get_display('קוראלציה בין חולים וסימפטומים')
    correlation = 'pearson: r={} p={}\nspearman: r={} p={}'.format(round(r_p, 3), round(p_p, 3), round(r_s, 3), round(p_s, 3))
    axes[0].set_title('{}\n{}'.format(title, correlation), loc='left')

    sm = plt.cm.ScalarMappable(cmap="Reds", norm=plt.Normalize(q_df.min(), q_df.max()))
    sm.set_array([])
    axes[0].get_legend().remove()
    axes[0].figure.colorbar(sm)
    
    # only for the minor axis
    axes[1].legend_.remove()
    
    # finishim
    plt.tight_layout()

    plt.savefig(os.path.join('figs', 'patients and symptoms correlation - scatter plot'))

In [ ]:
# find the best dates to take from the questionnaires
df = pd.DataFrame(columns=['start_date', 'end_date', 'delta_days', 'window_length', 'r_p', 'p_p', 'r_s', 'p_s'])

for delta_days in np.arange(1, 8):
    for window_length in np.arange(1, 8):

        end_date = patients_col[0] - pd.Timedelta(days=delta_days)
        start_date = end_date - pd.Timedelta(days=window_length-1)
        
        r_p, p_p, r_s, p_s = patients_symptoms_correlation_plot(symptoms, patients_col, start_date, end_date, corr_only=True)
        
        df.loc[df.shape[0]] = [start_date, end_date, delta_days, window_length, r_p, p_p, r_s, p_s]

In [ ]:
df.loc[(df['p_p'] < alpha) & (df['p_s'] < alpha)].sort_values('r_s', ascending=False)

In [ ]:
df.loc[df['p_s'] < alpha].sort_values('r_s', ascending=False)

In [ ]:
start_date = date(2020, 3, 26)
end_date = date(2020, 3, 28)

## scatter plot

In [ ]:
patients_symptoms_correlation_plot(symptoms, patients_col, start_date, end_date)

## bar plot

In [ ]:
q_df, p_df = patients_symptoms_correlation_plot(symptoms, patients_col, start_date, end_date, df_only=True)

top_q = q_df.sort_values(ascending=False).index

# q
plt.figure(figsize=(10, 6))
label = 'questionnaires prediction' if not hebrew else get_display('תחזית השאלונים')
g = sns.barplot(top_q, q_df.loc[top_q].values, color='lightgray', label=label)
g.set_ylabel('% symptoms in city\n{} - {}'.format(start_date, end_date)) if not hebrew else g.set_ylabel(get_display('אחוז הסימפטומים בעיר') + '\n{} - {}'.format(start_date, end_date))
# g.set_ylim(3, 8)

# p
g2 = g.twinx()
label = 'new patients' if not hebrew else get_display('חולים חדשים')
sns.scatterplot(top_q, p_df.loc[top_q].values, alpha=0.25, color='red', ax=g2)
rolling = p_df.loc[top_q].rolling(window=4).mean().fillna(method='bfill')
g2.plot(top_q, rolling.values, color='red', linewidth=4, label=label)
# g2.set_ylim(0, 0.75)
g2.set_ylabel('new patients per 1000 residences\n{} - {}'.format(patients_col[0], patients_col[1])) if not hebrew else g2.set_ylabel(get_display('חולים חדשים ל-1000 תושבים') + '\n{} - {}'.format(patients_col[0], patients_col[1]))

g.set_xticklabels(top_q, rotation=90) if not hebrew else g.set_xticklabels([get_display(label) for label in top_q], rotation=90)
g.set_xlabel('')

g2.figure.legend(bbox_to_anchor=(0.85, 0.9))

plt.tight_layout()
plt.savefig(os.path.join('figs', 'patients and symptoms correlation - bar plot'))

## follow ups

In [ ]:
# ??, ?? = patients_symptoms_correlation_plot(symptoms, ??, ??, ??, df_only=True)

# x = ??
# y = ??

# overlapped_citis = x.index.intersection(y.index)

# x = x.loc[overlapped_citis]
# y = y.loc[overlapped_citis]

# g = sns.scatterplot(x, y)

# r_p, p_p = pearsonr(x, y)
# r_s, p_s = spearmanr(x, y)
# correlation = 'pearson: r={} p={}\nspearman: r={} p={}'.format(round(r_p, 3), round(p_p, 3), round(r_s, 3), round(p_s, 3))

# g.set_title(correlation)
# g.set_xlabel('')
# g.set_ylabel('')

# plt.tight_layout()

# g.figure.savefig(os.path.join('figs', ''))

# symptoms time correlation

In [ ]:
start_date = date(2020, 3, 21)
end_date = date(2020, 4, 3)

In [ ]:
min_r = 0.8

In [ ]:
# to add possibility to use a list with more then one symptom

In [ ]:
# data - questions
q_df = questions_filtered_df[[location, time] + symptoms]
q_df = q_df[q_df[time] >= start_date]
q_df = q_df[q_df[time] <= end_date]
q_df = q_df.groupby([location]).filter(lambda x: len(np.unique(x[time])) == (end_date-start_date).days)

q_df = (q_df.groupby([location, time]).mean()*100).sort_index().reset_index(time)

g = None
plt.figure(figsize=(16, 5))
for city in np.unique(q_df.index):
    rolling = q_df.loc[city, symptoms[0]].rolling(window=3).mean()
    r, p = spearmanr(q_df.loc[city, time], rolling)
    if r > min_r and p < alpha:
        label = city if not hebrew else get_display(city)
        g = sns.lineplot(q_df.loc[city, time], rolling, label=label + ' (r={})'.format(round(r, 2)), marker='o', ax=g)
        
g.set_title('cities with highest correaltion of symptoms along time') if not hebrew else g.set_title(get_display('ערים עם הקוראלציה הכי גבוה עם הזמן'))
g.set_xlim(q_df.loc[city, time].min() + pd.Timedelta(days=3), q_df.loc[city, time].max())
g.set_xlabel('')

g.figure.savefig(os.path.join('figs', 'symptoms time correlation'))

# symptoms ratio between dates

In [ ]:
# data - questions
q_df = questions_filtered_df[[location, time] + symptoms]
q_df = q_df.groupby([location, time]).mean().reset_index()
q_df = q_df[q_df[time] >= date(2020, 3, 29)]
q_df = q_df[q_df[time] <= date(2020, 4, 3)]
q_df = q_df.groupby([location]).filter(lambda x: x.shape[0] == 6).sort_index().reset_index().drop('index', axis=1)

q_old_df = q_df
q_old_df = q_old_df[q_old_df[time] >= date(2020, 3, 29)]
q_old_df = q_old_df[q_old_df[time] <= date(2020, 3, 31)]
q_old_df = (q_old_df.drop(time, axis=1).groupby([location]).mean()*100).sort_index()

q_new_df = q_df
q_new_df = q_new_df[q_new_df[time] >= date(2020, 4, 1)]
q_new_df = q_new_df[q_new_df[time] <= date(2020, 4, 3)]
q_new_df = (q_new_df.drop(time, axis=1).groupby([location]).mean()*100).sort_index()

q_df = q_new_df.divide(q_old_df).sort_values(symptoms, ascending=False).reset_index()

plt.figure(figsize=(14, 6))
g = sns.barplot(location, symptoms[0], data=q_df, palette=(q_df[symptoms[0]] > 1).map({True: 'Red', False: 'Green'}).tolist())
g.axhline(1, color='black')
g.set_xticklabels(q_df[location], rotation=90) if not hebrew else g.set_xticklabels([get_display(label) for label in q_df[location]], rotation=90)
g.set_ylabel('ratio between symptoms') if not hebrew else g.set_ylabel(get_display('יחס הסימפטומים'))
g.set_xlabel('ratio between symptoms in')
title = '' if not hebrew else get_display('יחס הסימפטומים בין')
g.set_title(title + '\n{}-{}\n{}-{}'.format(date(2020, 4, 3), date(2020, 4, 1), date(2020, 3, 31), date(2020, 3, 29)))

plt.tight_layout()

g.figure.savefig(os.path.join('figs', 'ratio between symptoms'))

# symptoms over time

## per area

In [ ]:
def symptoms_patients_plot(**kwargs):

    # retract the data
    sub_data = kwargs.pop('data')
    
    q_data = sub_data[sub_data['variable'] != 'sick']
    if ~q_data.empty:
        sns.lineplot(time, 'value', marker='o', data=q_data, **kwargs)
            
    p_data = sub_data[sub_data['variable'] == 'sick']
    if ~p_data.empty:
        kwargs['color'] = 'red'
        sns.scatterplot(time, 'value', marker='*', s=300, data=p_data, **kwargs)

In [ ]:
# remove 'symptoms' from legend label

In [ ]:
# data - questions
q_df = questions_filtered_df[[area, location, time] + all_symptoms]
q_df = (q_df.groupby([area, location, time]).mean()*100).reset_index()

# data - patients
p_df = pd.DataFrame() # dismissing this part because of patients file change
# p_df = p_df[p_df[location].isin(np.unique(q_df[location]))]
# p_df = p_df[p_df[time] > q_df['date_only'].min()]
# p_df['variable'] = 'sick'
# p_df['value'] = -2.5

# plot
full_data = q_df.melt(id_vars=[area, location, time]) # pd.concat([q_df.melt(id_vars=[area, location, time]), p_df[[area, location, time, 'variable', 'value']]])
row = location
hue = 'variable'

for area_ in np.unique(full_data[area]):
    data = full_data[full_data[area] == area_]

    g = sns.FacetGrid(data=data,
                  row=row, hue=hue, 
                  sharex=False, sharey=True, 
                  xlim=(data[time].min(), data[time].max()), ylim=(-4, data['value'].max()),
                  height=3, aspect=4, size=None,
                  palette=None, legend_out=True,
                  row_order=np.unique(data[row].dropna()), hue_order=np.unique(data[hue].dropna()))

    g = g.map_dataframe(symptoms_patients_plot)

    g.set_titles(row_template='{row_name}', col_template='{col_name}')
    g.set_axis_labels(y_var='% positive answers') if not hebrew else g.set_axis_labels(y_var=get_display('% התשובות החיוביות'))
    g.add_legend()
#     for ax in g.axes.flatten():
#         ax.legend(bbox_to_anchor=(1, 1))
    
    title = 'symptoms over time with patients in {}'.format(area_) if not hebrew else get_display('סימפטומים לאורך זמן עם חולים') + ' {}'.format(area_)

    g.savefig(os.path.join('figs', 'symptoms over time with patients in {}'.format(area_)))

## entire country

In [ ]:
# data - questions
q_df = questions_filtered_df[[time] + symptoms].sort_values(time).groupby([time]).mean().rolling(window=5).mean().reset_index()

plt.figure(figsize=(14, 6))
g = sns.lineplot(q_df[time], q_df[symptoms[0]], marker='o')
g.set_ylabel('average symptoms') if not hebrew else g.set_ylabel(get_display('סימפטומים ממוצעים'))
g.set_xlabel('')
g.set_title('average symptoms across the country') if not hebrew else g.set_title(get_display('סימפטומים ממוצעים בכל הארץ'))

plt.tight_layout()

g.figure.savefig(os.path.join('figs', 'country symptoms'))

## high cities

In [ ]:
# get cities automatically

In [ ]:
# data - questions
q_df = questions_filtered_df[questions_filtered_df['city_en'].isin(['ASHQELON',"BE'ER SHEVA",'BET SHEMESH','BENE BERAQ',"MODI'IN ILLIT",'PETAH TIQWA'])]
q_df = q_df[[location, time] + symptoms].sort_values([location, time]).groupby([location, time]).mean().rolling(window=5).mean().reset_index()

plt.figure(figsize=(14, 6))
g = sns.lineplot(q_df[time], q_df[symptoms[0]], hue=q_df[location], marker='o')
g.set_ylabel('average symptoms') if not hebrew else g.set_ylabel(get_display('סימפטומים ממוצעים'))
g.set_xlabel('')
g.set_title('cities with high symptoms') if not hebrew else g.set_title(get_display('ערים עם סימפטומים גבוהים'))

handels, labels = g.get_legend_handles_labels()
for i in np.arange(len(labels)):
    labels[i] = get_display(labels[i])
g.legend(handels, labels, title='')

plt.tight_layout()

g.figure.savefig(os.path.join('figs', 'high cities symptoms'))

# questionnaires over time

In [ ]:
name = 'questionnaires' if not hebrew else get_display('שאלונים')

# data
q_df = questions_df.groupby(time).apply(len).reset_index().rename(columns={0: name})

# plot
plt.figure()
pal = sns.color_palette('Greens', len(data))
g = sns.barplot(x=time, y=name, data=q_df, palette=np.array(pal)[q_df[name].argsort().argsort()])

g.set_xticklabels(g.get_xticklabels(), rotation=90)
for index, label in enumerate(g.get_xticklabels()):
    if index % 2 != 0:
        label.set_visible(False)
g.set_xlabel('')

g.set_title('questionnaires over time') if not hebrew else g.set_title(get_display('שאלונים לאורך הזמן')) 
plt.tight_layout()
g.figure.savefig(os.path.join('figs', 'questionnaires over time'))

# symptoms per location

In [ ]:
# data
data = questions_filtered_df[[location] + all_symptoms]
data = (data.groupby([location]).mean()*100).reset_index()

symptoms_by_order = data[all_symptoms].sum().sort_values(ascending=False).index.tolist()
location_by_order = data.set_index(location)[all_symptoms].sum(axis=1).sort_values(ascending=False).index.tolist()

# Make the PairGrid
g = sns.PairGrid(
    data=data.set_index(location).loc[location_by_order].reset_index(),
    x_vars=symptoms_by_order, y_vars=location,
    height=10, aspect=.25)

# Draw a dot plot using the stripplot function
g.map(sns.stripplot, size=10, orient='h', palette="ch:s=1,r=-.1,h=1_r", linewidth=1, edgecolor='w')

# Use the same x axis limits on all columns and add better labels
g.set(xlim=(0, 25), xlabel='% positive answers', ylabel='') if not hebrew else g.set(xlim=(0, 25), xlabel=get_display('אחוז התשובות החיוביות'), ylabel='')

for ax, title in zip(g.axes.flat, symptoms_by_order):

    # Set a different title for each axes
    ax.set(title=title)

    # Make the grid horizontal instead of vertical
    ax.xaxis.grid(False)
    ax.yaxis.grid(True)

sns.despine(left=True, bottom=True)

title = 'symptoms per location' if not hebrew else get_display('סימפטומים לפי מיקום')
plt.subplots_adjust(top=0.9)
g.fig.suptitle(title)
g.savefig(os.path.join('figs', 'symptoms per location'))

# symptoms over conditions

In [ ]:
# data - q
q_df = questions_df # intinatinally not filtered because its without time
q_df = q_df[[location] + all_symptoms + all_conditions]
q_df = (q_df.groupby([location]).mean()*100).reset_index()

symptoms_by_order = q_df[all_symptoms].sum().sort_values(ascending=False).index.tolist()
conditions_by_order = q_df[all_conditions].sum().sort_values().index.tolist()

# plot
g = sns.pairplot(q_df, x_vars=symptoms_by_order, y_vars=conditions_by_order, hue=location)

title = 'symptoms over conditions' if not hebrew else get_display('סימפטומים מול מצבים רפואיים')
plt.subplots_adjust(top=0.9)
g.fig.suptitle(title)

g.savefig(os.path.join('figs', 'symptoms over conditions'))

# MOH patients file combiner

In [ ]:
# reading and cleaning MOH patients files
f2703 = '/net/mraid08/export/jafar/Microbiome/Analyses/saar/corona/MOH/originals/0327.xlsx'
f2703 = pd.read_excel(f2703).dropna().rename(columns={'patients': date(2020, 3, 27)}) # does not have population column
f2703['city_he'] = [get_display(city) for city in f2703['city_he']]

f3103 = '/net/mraid08/export/jafar/Microbiome/Analyses/saar/corona/MOH/originals/0331.xlsx'
f3103 = pd.read_excel(f3103).dropna().rename(columns={'patients': date(2020, 3, 31)}).drop('population', axis=1)

f0104 = '/net/mraid08/export/jafar/Microbiome/Analyses/saar/corona/MOH/originals/0401.xlsx'
f0104 = pd.read_excel(f0104).dropna().rename(columns={'patients': date(2020, 4, 1)}).drop('population', axis=1)

f0304 = '/net/mraid08/export/jafar/Microbiome/Analyses/saar/corona/MOH/originals/0403.xlsx'
f0304 = pd.read_excel(f0304).dropna().rename(columns={'patients': date(2020, 4, 3)}).drop('population', axis=1)
f0304['city_he'] = [get_display(city) for city in f0304['city_he']]

f0504 = '/net/mraid08/export/jafar/Microbiome/Analyses/saar/corona/MOH/originals/0405.xlsx'
f0504 = pd.read_excel(f0504).dropna().rename(columns={'patients': date(2020, 4, 5)}).drop('population', axis=1)
f0504['city_he'] = [get_display(city) for city in f0504['city_he']]

f0604 = '/net/mraid08/export/jafar/Microbiome/Analyses/saar/corona/MOH/originals/0406.xlsx'
f0604 = pd.read_excel(f0604).dropna().rename(columns={'patients': date(2020, 4, 6)}).drop('population', axis=1)
f0604['city_he'] = [get_display(city) for city in f0604['city_he']]

f0704 = '/net/mraid08/export/jafar/Microbiome/Analyses/saar/corona/MOH/originals/0407.xlsx'
f0704 = pd.read_excel(f0704).dropna().rename(columns={'patients': date(2020, 4, 7)}) # this has the longest population column
f0704['city_he'] = [get_display(city) for city in f0704['city_he']]

patients_MOH_df = f2703.set_index('city_he').join(f3103.set_index('city_he'), how='outer').reset_index()
patients_MOH_df = patients_MOH_df.set_index('city_he').join(f0104.set_index('city_he'), how='outer').reset_index()
patients_MOH_df = patients_MOH_df.set_index('city_he').join(f0304.set_index('city_he'), how='outer').reset_index()
patients_MOH_df = patients_MOH_df.set_index('city_he').join(f0504.set_index('city_he'), how='outer').reset_index()
patients_MOH_df = patients_MOH_df.set_index('city_he').join(f0604.set_index('city_he'), how='outer').reset_index()
patients_MOH_df = patients_MOH_df.set_index('city_he').join(f0704.set_index('city_he'), how='outer').reset_index()
patients_MOH_df['city_en'] = patients_MOH_df['city_he'].map(city_he2en_dict)
# patients_MOH_df.to_csv(os.path.join('MOH', 'MOH_combined.csv'))